# MLOps Driven Training in 10 Easy Steps
## Step 1:  Access Your Machine Learning Workspace - We will use the Azure ML Python SDK

In [23]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.7.0


In [10]:

import os

# Give your worjspace details
subscription_id = os.getenv("SUBSCRIPTION_ID",default="")
resource_group = os.getenv("RESOURCE_GROUP", default="")
workspace_name = os.getenv("WORKSPACE_NAME", default="")
workspace_region = os.getenv("WORKSPACE_REGION", default="")

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

# write the details of the #workspace to a configuration file to the notebook library
ws.write_config()

In [3]:
from azureml.core import Workspace


# You can read directly from the config file going forward
ws = Workspace.from_config()


## Step 2: Access Your Compute Target in your ML Workspace

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpu-cluster


In [7]:
# You can access the compute targets associated with the workspace as shown below
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

cvcluster AKS Succeeded
akscpu AKS Succeeded
myaks AKS Succeeded
aks-mlops AKS Succeeded
akscompute AKS Succeeded
aks-mnist AKS Succeeded
gpucluster AmlCompute Succeeded
cpucluster AmlCompute Succeeded
nc6cluster AmlCompute Succeeded
cpucompute ComputeInstance Succeeded


## Step 3: Get your training script ready - Track Runs against Dataset, Metrics and Outputs
1. Include a run time argument for data folder. Other arguments as necessary for your script.
2. Include Metrics Logging
3. Write your Outputs to './outputs' folder

In [8]:

with open('./train.py', 'r') as f:
    print(f.read())

import os
import sys
import argparse
import joblib
import pandas as pd

from azureml.core import Run
from azureml.core.run import Run

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


def getRuntimeArgs():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-path', type=str)
    args = parser.parse_args()
    return args


def main():
    args = getRuntimeArgs()
    run = Run.get_context()

    credit_data_df = pd.read_csv(os.path.join(args.data_path, 'german_credit_data.csv'))

    #credit_data_df = pd.read_csv(os.path.join(run.input_datasets['data'], 'german_credit_data.csv'))

    

    clf = model_train(credit_data_df, run)

    #copying to "o

## Step 4: Create a ML Experiment in your workspace to track your Runs

In [24]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='mlops-demo')

## Step 5: Run your Experiment Locally(Optional)

Estimator objects help to bundle the run configurations required for your Run like the data, source code, dependencies and other parameters

In [25]:
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails


# use a conda environment on local computer

script_params = {
    '--data-path': '/home/azureuser/cloudfiles/code/Users/meranjit/MLOps_Estimators/sample-data'
}

est = Estimator(source_directory='.', script_params=script_params, compute_target='local', entry_script='train.py', use_docker=False, conda_packages=['scikit-learn', 'joblib'])
run = exp.submit(est)
RunDetails(run).show()

WARNING - 'use_docker' parameter will be deprecated. Please use 'environment_definition' instead.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [18]:
run.cancel()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Step 6: Register a Datastore
Datastore stores the connection strings for the supported Azure Storage

In [7]:
from azureml.core import Datastore
blob_datastore_name='mlopsdatastore' # Name of the datastore to workspace
container_name=os.getenv("BLOB_CONTAINER", "training-data") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "mlblobdatastore") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", "8RyWjAWGoH1Qx6oT0KzOI0ffILJXB4i9i1gFPAUvTM7rm3dDkVmdeifmDBHkpXa5QOwV0iDVDa63Q/gXV6K6Qg==") # Storage account access key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [12]:
# List all datastores registered in the current workspace
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)


azureml_globaldatasets AzureBlob
batchscoring AzureBlob
mlopsdatastore AzureBlob
mnist_datastore AzureBlob
tfworld AzureBlob
stackoverflowstore AzureBlob
stackoverflow_ds AzureBlob
newsdata AzureBlob
animalsdemostore AzureBlob
animalsstore AzureBlob
petsstore AzureBlob
logsds AzureBlob
toysds AzureBlob
logos AzureBlob
animals AzureBlob
lesion_inference_datastore AzureBlob
isic2018 AzureBlob
mlblobdatastore AzureBlob
mlblobdatastore__frauddetectionembeddings AzureBlob
mlblobdatastore__frauddetection AzureBlob
frauddetectionds AzureBlob
uploadedresources AzureBlob
livenessdataset AzureBlob
mlblobdatastore__wisconsin AzureBlob
mlblobdatastore__liver AzureBlob
azurecognitivesearch__basicdemo AzureBlob
azurecognitivesearch__powerbi AzureBlob
mlblobdatastore__dataset AzureBlob
mlserviceworks7462121151__azureml AzureBlob
smtprodwestus21globaluploadedresources AzureBlob
commandjson AzureBlob
dataset AzureBlob
fake AzureBlob
real AzureBlob
images_datastore AzureBlob
kineticsdsmodel AzureBlob
ki

In [26]:
# Get a named datastore from the current workspace
from azureml.core import Datastore
datastore = Datastore.get(ws, datastore_name='mlopsdatastore')

## Step 7: Register a Dataset

Datasets helps to manage your dataset versions, manage security controls on your dataset and help access dataset in machine learning friendly formats like pandas dataframe, spark dataframe etc



In [27]:
from azureml.core import Dataset
dataset = Dataset.File.from_files(path = [(datastore, 'data')])

# see a list of files referenced by dataset
dataset.to_path()

['/german_credit_data.csv']

In [10]:
mlops_ds = dataset.register(workspace=ws,
                                 name='mlops_ds',
                                 description='mlops training data',
                                 create_new_version=False)

In [28]:
registered_ds = ws.datasets['mlops_ds']

In [27]:
registered_ds

{
  "source": [
    "('mlopsdatastore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "56707610-063f-435c-b04a-1177e79bf397",
    "name": "mlops_ds",
    "version": 1,
    "description": "mlops training data",
    "workspace": "Workspace.create(name='mlserviceworkspace', subscription_id='601f4351-33bb-4d76-96ca-886940409b3d', resource_group='mlserviceworkspace')"
  }
}

## Step 8 : Manage your dependencies with Environments

In [29]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
myenv = Environment(name="mlopsenv")
conda_dep = CondaDependencies()


conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("joblib")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep
#myenv.save_to_directory('env')

In [47]:
# Register the environment 
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "mlopsenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
          

In [30]:
restored_environment = Environment.get(workspace=ws,name="mlopsenv",version="1")

In [ ]:
Environment.list(workspace=ws) 

## Step 9: Submit Run Remotely on AML compute cluster

Estimator objects help to bundle the run configurations required for your Run like the data, source code, dependencies and other parameters

In [31]:
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails


# use a conda environment, don't use Docker, on local computer

script_params = {
    '--data-path': dataset.as_named_input('data').as_mount()
}

est = Estimator(source_directory='.', script_params=script_params, compute_target=cpu_cluster, entry_script='train.py', environment_definition =  myenv )
run = exp.submit(est)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Step 10: Register the Model against the Dataset

In [32]:

# register the best model with the input dataset
model = run.register_model(model_name='german-credit-model', model_path=os.path.join('outputs', 'model.pkl'),
                           datasets =[('training data',registered_ds)])

In [17]:
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive import choice
param_sampling = GridParameterSampling( {
        "cvalue": choice(0.001, 0.01, 0.1, 1, 10, 100, 1000)
    }
)

In [18]:
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [19]:
script_params = {
    '--data-path': dataset.as_named_input('data').as_mount()
}

est = Estimator(source_directory='.', script_params=script_params, compute_target=cpu_cluster, entry_script='train_hyperdrive.py', environment_definition =  myenv )


In [20]:
hdc = HyperDriveConfig(estimator=est, 
                       hyperparameter_sampling=param_sampling, 
                       policy=policy, 
                       primary_metric_name="Test accuracy",
                       primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                       max_total_runs=2,
                       max_concurrent_runs=2)

In [21]:
hdr = exp.submit(config=hdc)

In [22]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [51]:
best_run = hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--data-path', 'DatasetConsumptionConfig:data', '--cvalue', '0.01']


In [ ]:
# register the best model with the input dataset
model = best_run.register_model(model_name='german-credit-model', model_path=os.path.join('outputs', 'model.pkl'),
                           datasets =[('training data',registered_ds)])